In [2]:
import os

import numpy as np
import pandas as pd

# ---
run_prefix = "meanAnn_basalt_shortRun_noFert_fixedRate"  # prefix of output run names
# base_climdir = "/home/tykukla/aglime-swap-cdr/scepter/clim-inputs"
# clim_general = "cesm2le_monthly_ens-1231-001"   # identifier for climate files to use
clim_ts = "NA"  # "1950-2020"
const_dict = {
    "duration": 70,  # duration of run (starts from earliest year)
    "singlerun_seasonality": False,
    "dustsp": "gbas",
    "dustrad": 150,
    # "nitrif": 10,    # [gN/m2/yr] (10 is ~ 90 lbs/acre/yr; 24.6 ~ 220 lbs/acre/year)
    "add_secondary": True,
    "climatedir": "NA",  # os.path.join(base_climdir, clim_general, clim_ts),
}
# ---
savefn = run_prefix + "_v0.csv"

# ---
# split into multiple runs?
multi_run_split = False
max_iters_per_set = 20

In [3]:
# --- set the dust rate array
arr1 = np.arange(0, 1, 1)  # np.arange(0, 1, 0.2)
arr2 = np.arange(1, 10, 10)  # np.arange(1,10, 1)
arr3 = np.arange(10, 23, 10)  # np.arange(10, 23, 4)
dustflx_tonha = np.concatenate((arr1, arr2, arr3))
dustflx_in = dustflx_tonha * 100  # convert to g m-2
dustflx_in

array([   0,  100, 1000, 2000])

In [4]:
# --- read in site inputs
spin_in_path = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"
spin_inputs = "spinup-inputs.csv"
# read in
df = pd.read_csv(os.path.join(spin_in_path, spin_inputs))
df

,site,spinname,lat,lon,mat,soilmoisture,qrun,tsom,erosion,nitrif,tph,cec,tec,tsoilco2,poro,alpha
0,site_311,site_311,42.5,-91,8.22219,0.282727,0.351361,2.051667,0.001013,1.005952,6.058007,21.10329,20.98031,-1.80371,0.447,2.0
1,site_411,site_411,32.0,-83,18.52789,0.231552,0.243426,2.276667,0.000840,0.831883,5.200242,1.96125,46.91557,-1.61194,0.419,2.0


In [5]:
# define the vars to collect
allvars = [
    "duration",
    "singlerun_seasonality",
    "cec",
    "dustsp",
    "dustrate",
    "dustrad",
    "add_secondary",
    "spinrun",
    "newrun_id",
    "climatedir",
]

In [6]:
def newrun_id_fxn(pref, site, clim_tag, dstflx):
    # return pref + "_" + site + "_" + clim_tag + "_app-" + dstflx
    return pref + "_" + site + "_app-" + dstflx

In [7]:
# --- functions to derive inputs
def thisrow(tdf, const_dict, allvars, run_prefix, clim_ts, dstflx_fn, dstflx):
    outarr = []  # save row
    thissite = tdf["site"]
    for var in allvars:
        if var == "spinrun":
            outarr.append(thissite + "_spintuneup")
        elif var == "newrun_id":
            outarr.append(newrun_id_fxn(run_prefix, thissite, clim_ts, dstflx_fn))
        elif var == "climatefiles":
            outarr.append(thissite)
        else:  # var is defined by const_dict or df
            tvar = const_dict.get(var)
            if tvar is None:
                tvar = tdf[var]
                outarr.append(tvar)
            else:
                outarr.append(tvar)
    return outarr

In [8]:
# --- loop through sites
# create output pandas df
outdf = pd.DataFrame(columns=allvars)
# loop
cnt = 0
for rundx in range(len(dustflx_in)):
    for idx in range(len(df)):
        tdf = df.loc[idx]
        # get dust flux for filename and table
        dstflx_fn = str(dustflx_tonha[rundx].round(2)).replace(".", "p")
        dstflx = dustflx_in[rundx].round(2)
        # add to const dict
        const_dict.update({"dustrate": dstflx})
        thesevars = thisrow(
            tdf, const_dict, allvars, run_prefix, clim_ts, dstflx_fn, dstflx
        )
        outdf.loc[cnt] = thesevars
        cnt += 1
outdf

,duration,singlerun_seasonality,cec,dustsp,dustrate,dustrad,add_secondary,spinrun,newrun_id,climatedir
0,70,False,21.10329,gbas,0,150,True,site_311_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
1,70,False,1.96125,gbas,0,150,True,site_411_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
2,70,False,21.10329,gbas,100,150,True,site_311_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
3,70,False,1.96125,gbas,100,150,True,site_411_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
4,70,False,21.10329,gbas,1000,150,True,site_311_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
5,70,False,1.96125,gbas,1000,150,True,site_411_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
6,70,False,21.10329,gbas,2000,150,True,site_311_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA
7,70,False,1.96125,gbas,2000,150,True,site_411_spintuneup,meanAnn_basalt_shortRun_noFert_fixedRate_site_...,NA


In [9]:
outdf["newrun_id"][4]

'meanAnn_basalt_shortRun_noFert_fixedRate_site_311_app-10'

In [10]:
savepath_batch = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"

if multi_run_split:
    # calculate the number of chunks needed
    num_chunks = -(-len(outdf) // max_iters_per_set)  # round up the division result
    # splitting df into chunks of approximately 20 rows each
    outdfs = [
        outdf.iloc[i * max_iters_per_set : (i + 1) * max_iters_per_set]
        for i in range(num_chunks)
    ]
    # saving each df to a different file
    for i, df_chunk in enumerate(outdfs):
        tot_dfs = len(outdfs)
        savefn_nosuff = savefn.rstrip(".csv")
        savefn_out = f"{savefn_nosuff}_set{i + 1}of{tot_dfs}.csv"
        print(os.path.join(savepath_batch, savefn_out))
        df_chunk.to_csv(os.path.join(savepath_batch, savefn_out), index=False)
        # also save total
        outdf.to_csv(os.path.join(savepath_batch, savefn), index=False)
else:
    savepath_batch = "/home/tykukla/aglime-swap-cdr/scepter/batch-inputs"
    outdf.to_csv(os.path.join(savepath_batch, savefn), index=False)

In [ ]:
# -----------------